In [ ]:
# import tensorflow as tf

# print(tf.__version__)

In [ ]:
# import os
# os.environ.get("tf_upgrade_v2")

In [ ]:
# !tf_upgrade_v2 \
#   --infile /home/zcxia/TradeMaster/data_synthesis/TimeGAN/timegan.py \
#   --outfile /home/zcxia/TradeMaster/data_synthesis/TimeGAN/timegan_v2.py \
#   --reportfile report.txt

In [ ]:
# import tensorflow as tf

# print(tf.__version__)

In [ ]:
# import os
# os.environ.get("tf_upgrade_v2")

# TimeGAN Tutorial

## Time-series Generative Adversarial Networks

- Paper: Jinsung Yoon, Daniel Jarrett, Mihaela van der Schaar, "Time-series Generative Adversarial Networks," Neural Information Processing Systems (NeurIPS), 2019.

- Paper link: https://papers.nips.cc/paper/8789-time-series-generative-adversarial-networks

- Last updated Date: April 24th 2020

- Code author: Jinsung Yoon (jsyoon0823@gmail.com)

This notebook describes the user-guide of a time-series synthetic data generation application using timeGAN framework. We use Stock, Energy, and Sine dataset as examples.

### Prerequisite
Clone https://github.com/jsyoon0823/timeGAN.git to the current directory.

## Necessary packages and functions call

- timegan: Synthetic time-series data generation module
- data_loading: 2 real datasets and 1 synthetic datasets loading and preprocessing
- metrics: 
    - discriminative_metrics: classify real data from synthetic data
    - predictive_metrics: train on synthetic, test on real
    - visualization: PCA and tSNE analyses

In [47]:
## Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import warnings
warnings.filterwarnings("ignore")

# 1. TimeGAN model
from timegan import timegan
# 2. Data loading
from data_loading import real_data_loading, sine_data_generation
# 3. Metrics
from metrics.discriminative_metrics import discriminative_score_metrics
from metrics.predictive_metrics import predictive_score_metrics
from metrics.visualization_metrics import visualization
import pandas as pd
from tqdm import tqdm
import pickle
import importlib
# importlib.reload(styletimegan)
%load_ext autoreload
%autoreload 2

In [ ]:
def get_intervals(data):
    index=data['index']
    last_value=index[0]-1
    last_index=0
    intervals=[]
    for i in range(data.shape[0]):
        if last_value!=index[i]-1:
            intervals.append([last_index,i])
            last_value=index[i]
            last_index=i
        last_value=index[i]
    intervals.append([last_index, i])
    return intervals

In [ ]:
def interpolation(data):
    max_len=24
    l=len(data)
    to_fill=max_len-l
    if to_fill!=0:
        interval=max_len//to_fill
        for j in range(to_fill):
            idx=(interval+1)*j+interval
            data.insert(min(idx,len(data)-1),float('nan'))
    data=pd.Series(data).interpolate(method='polynomial', order=2)
    return data

In [ ]:
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    norm_data = numerator / (denominator + 1e-7)
    return np.min(data, 0),np.max(data, 0),norm_data

In [ ]:
def minmax_normlization(data):
    normalized_data=(data-data.min())/(data.max()-data.min())
    return normalized_data

In [ ]:
def get_data_of_same_length(data,seq_len):
    data_processed=[]
    intervals=get_intervals(data)
    temp_data=[]
    data.drop(columns=['index'])
    for interval in intervals:
#         print(interval)
        data_seg=data.iloc[interval[0]:interval[1],:]
        for i in range(0, len(data_seg) - seq_len):
            _x = data_seg.iloc[i:i + seq_len,:]
            temp_data.append(_x)
    return temp_data

# Run through

In [ ]:
def dataset_prepareation(path):
    data=pd.read_csv(path).reset_index()
    tics=data['tic'].unique()
    # features=[ 'open', 'high', 'low', 'close', 'adjcp','zopen', 'zhigh', 'zlow', 'zadjcp', 'zclose', 'zd_5', 'zd_10',
    #    'zd_15', 'zd_20', 'zd_25', 'zd_30', 'pct_return', 'adjcp_filtered',
    #    'pct_return_filtered','volume']
    features=['open','high','low','close','adjcp','volume']
    ret=[]
    for col in data.columns:
        if col in features:
            ret.append(col)
    features=ret
    min_sclar_by_tic={}
    max_sclar_by_tic={}
    for tic in tics:
        data_by_tic=data.loc[data['tic']==tic,features].astype(float)
        min_scalr,max_sclar,norm_data_by_tic=MinMaxScaler(data_by_tic)
        # print(min_scalr.shape)
        data.loc[data['tic']==tic,features]=norm_data_by_tic
        min_sclar_by_tic[tic]=min_scalr
        max_sclar_by_tic[tic]=max_sclar
    stock_group_num=len(data['stock_type'].unique())
    regime_num=len(data['label'].unique())
    for tic in tics:
        with open('./data/scalr/'+str(tic)+'_minsclar'+'.pickle', 'wb') as handle:
            pickle.dump(min_sclar_by_tic[tic], handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('./data/scalr/'+str(tic)+'_maxsclar'+'.pickle', 'wb') as handle:
            pickle.dump(max_sclar_by_tic[tic], handle, protocol=pickle.HIGHEST_PROTOCOL)
        for j in range(regime_num):
            data_seg=data.loc[(data['tic']==tic) & (data['label']==j),['index','open','high','low','close','adjcp','volume']]
    #         data_dict[(i,j)]=data_seg
            data_seg.to_csv('./data/data_seg_'+tic+'_'+str(j)+'.csv')
        

## Data Loading

Load original dataset and preprocess the loaded data.

- data_name: stock, energy, or sine
- seq_len: sequence length of the time-series data

In [ ]:
# data=pd.read_csv('/home/hcxia/TradeMaster_dev/TradeMaster/data_synthesis/TimeGAN/GOOG_labeled_3_24.csv').reset_index()

In [ ]:
# data=pd.read_csv("./DJI_all_labeled_3_24.csv").reset_index()

In [ ]:
# dataset_prepareation("/home/hcxia/TradeMaster_dev/TradeMaster/data/data/other/DJI_all_labeled_3_24.csv")

In [ ]:
# dataset_prepareation('/home/hcxia/TradeMaster_dev/TradeMaster/data/data/other/GOOG_labeled_3_24.csv')

# preprocess data by stock group and regime

Normalize features with in own tic

### TimeGan Dataset description:
Stocks. By contrast, sequences of stock prices are continuous-valued but aperiodic; furthermore,
features are correlated with each other. We use the daily historical Google stocks data from 2004 to
2019, including as features the volume and high, low, opening, closing, and adjusted closing prices.

# Make data dict

In [ ]:
def prepare_data_for_trainning(path):
    data=pd.read_csv(path).drop('index', axis=1)
    data=data.reset_index().rename(columns={data.index.name:'index'})
#     plt.plot(data['adjcp'])
#     plt.show()
#     display(data.head())
    data.to_csv('temp_data.csv')
    data=get_data_of_same_length(data,24)
#     for i in range(20):
#         plt.plot(data[i]['adjcp'])
#         plt.show()
#         print(i)
#         display(data[i])
#     display(len(data))
    data=[d.loc[:,['open','high','low','close','adjcp','volume']].to_numpy() for d in data]
    return data

### GOOG data

In [ ]:
GOOG_data={}
for i in range(3):
    GOOG_data['data_seg_GOOG_'+str(i)]=prepare_data_for_trainning('./data/data_seg_GOOG_'+str(i)+'.csv')
    print(i,len(GOOG_data['data_seg_GOOG_'+str(i)]))

### Single stock data

In [ ]:
data=pd.read_csv("../../data/data/other/DJI_all_labeled_3_24.csv").reset_index()
tics=data['tic'].unique()
data_dict_tic={}
for tic in tics:
    data_dict_tic[tic]={}
    for i in range(3):
        data_dict_tic[tic]['data_seg_'+str(tic)+'_'+str(i)]=prepare_data_for_trainning('./data/data_seg_'+str(tic)+'_'+str(i)+'.csv')
        print(tic,i,len(data_dict_tic[tic]['data_seg_'+str(tic)+'_'+str(i)]))

### Stock group data

In [ ]:
data=pd.read_csv("../../data/data/other/DJI_all_labeled_3_24.csv").reset_index()
# display(data.columns)
tic_group_pair=data.loc[:,['tic','stock_type']]
tic_group_pair=tic_group_pair.groupby(['tic','stock_type']).size().reset_index(name='Freq')
stock_group_num=len(data['stock_type'].unique())
tic_in_group={}
for group in range(stock_group_num):
#     if group not in tic_in_group：
#         tic_in_group[group]=[]
    tic_in_group[group]=list(tic_group_pair.loc[tic_group_pair['stock_type']==group,:]['tic'].unique())

In [ ]:
print(tic_in_group)

In [ ]:
data=pd.read_csv("../../data/data/other/DJI_all_labeled_3_24.csv").reset_index()
stock_group_num=len(data['stock_type'].unique())
data_dict_group={}
for group in range(stock_group_num):
    data_dict_group[group]={}
    for i in range(3):
        if 'data_seg_'+str(group)+'_'+str(i) not in data_dict_group[group]:
            data_dict_group[group]['data_seg_'+str(group)+'_'+str(i)]=[]
        for tic in tic_in_group[group]:
            data_dict_group[group]['data_seg_'+str(group)+'_'+str(i)].extend(data_dict_tic[tic]['data_seg_'+str(tic)+'_'+str(i)])
        print(group,i,len(data_dict_group[group]['data_seg_'+str(group)+'_'+str(i)]))

### All dji stock data

In [ ]:
data_all={}
for i in range(3):
    if 'data_seg_'+'all'+'_'+str(i) not in data_all:
        data_all['data_seg_'+'all'+'_'+str(i)]=[]
    for tic in tics:
        data_all['data_seg_'+'all'+'_'+str(i)].extend(data_dict_tic[tic]['data_seg_'+str(tic)+'_'+str(i)])
    print(i,len(data_all['data_seg_'+'all'+'_'+str(i)]))

## Training helper

# data set:

- GOOG_data
- data_dict_tic (dict of dict by tic)
- data_dict_group (dict of dict by group num)
- data_all

## Set network parameters

TimeGAN network parameters should be optimized for different datasets.

- module: gru, lstm, or lstmLN
- hidden_dim: hidden dimensions
- num_layer: number of layers
- iteration: number of training iterations
- batch_size: the number of samples in each batch

In [ ]:
print(GOOG_data.keys())
print(data_dict_tic.keys())
print(data_dict_group.keys())
print(data_all.keys())

In [ ]:
def training(data_set_dict):
    parameters = dict()
    parameters['module'] = 'gru' 
    parameters['hidden_dim'] = 24
    parameters['num_layer'] = 3
    parameters['iterations'] = 10000
    parameters['batch_size'] = 128
    gnerated_dataset_dict={}
    for dataset_name, data_set in data_set_dict.items():
        print(str(dataset_name))
        gnerated_dataset_dict[dataset_name] = timegan(data_set, parameters,device=1,save_name=str(dataset_name))   
    return gnerated_dataset_dict

In [ ]:
data_dict_tic_generated={}
for tic in data_dict_tic.keys():
    if tic not in data_dict_tic_generated:
        data_dict_tic_generated[tic]=training(data_dict_tic[tic])

In [ ]:
GOOG_data_generated=training(GOOG_data)

data_seg_GOOG_0




Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/te

2023-01-25 20:55:02.859944: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-01-25 20:55:02.919303: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 3090 major: 8 minor: 6 memoryClockRate(GHz): 1.695
pciBusID: 0000:3b:00.0
2023-01-25 20:55:02.919443: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory
2023-01-25 20:55:02.919490: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory
2023-01-25 20:55:02.919534: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcufft.so.10.0'; dlerror: libcufft.so.10.0: cannot open shared 



Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [ ]:
data_dict_tic_APPL_generated=training(data_dict_tic['AAPL'])
data_dict_group_3_generated=training(data_dict_group[3])
GOOG_data_generated=training(GOOG_data)
data_all_generated=training(data_all)

In [ ]:
data_dict_group_4_generated=training(data_dict_group[4])

## Save generated data

In [ ]:
data_to_save=[
data_dict_tic_APPL_generated,
data_dict_group_3_generated,
GOOG_data_generated,
data_all_generated,
data_dict_group_4_generated
]
for tic in data_dict_tic_generated.keys():
    locals()['data_dict_tic_'+tic+'_generated']=data_dict_tic_generated[tic]
    data_to_save.append(locals()['data_dict_tic_'+tic+'_generated'])

In [ ]:
import pickle
import inspect

    
def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]
for i in range(len(data_to_save)):
    try:
        with open('./generated_data/'+retrieve_name(data_to_save[i])[-1]+'.pickle', 'wb') as handle:
            pickle.dump(data_to_save[i], handle, protocol=pickle.HIGHEST_PROTOCOL)
        print('writen to '+retrieve_name(data_to_save[i])[-1]+'.pickle')
    except:
        print('skip this')

## Evaluate the generated data



In [ ]:
def dis_score(ori_data, generated_data):
    metric_iteration = 5
    discriminative_score = list()
    for _ in range(metric_iteration):
        temp_disc = discriminative_score_metrics(ori_data, generated_data)
        discriminative_score.append(temp_disc)
    print('Discriminative score: ' + str(np.round(np.mean(discriminative_score), 4)))
    return np.round(np.mean(discriminative_score), 4)

def pred_score(ori_data, generated_data):
    predictive_score = list()
    metric_iteration = 5
    for tt in range(metric_iteration):
        temp_pred = predictive_score_metrics(ori_data, generated_data)
        predictive_score.append(temp_pred)   
    print('Predictive score: ' + str(np.round(np.mean(predictive_score), 4)))
    return np.round(np.mean(predictive_score), 4)
    
def evaluation(data_set_dict,genrated_data_set_dict):
    length_dict={}
    discriminative_score_dict={}
    predictive_score_dict={}
    for dataset_name, data_set in data_set_dict.items():
        length_dict[dataset_name]=len(data_set_dict[dataset_name])
        discriminative_score_dict[dataset_name] = dis_score(data_set_dict[dataset_name],genrated_data_set_dict[dataset_name])
        predictive_score_dict[dataset_name] = pred_score(data_set_dict[dataset_name],genrated_data_set_dict[dataset_name])
        visualization(data_set_dict[dataset_name],genrated_data_set_dict[dataset_name], 'pca')
        visualization(data_set_dict[dataset_name],genrated_data_set_dict[dataset_name], 'tsne')
    return length_dict,discriminative_score_dict,predictive_score_dict

In [ ]:
data_dict_tic_res={}
for tic in data_dict_tic.keys():
    data_dict_tic_res[tic]=[evaluation(data_dict_tic[tic],data_dict_tic_generated[tic])]

In [ ]:
data_dict_tic_APPL_generated=training(data_dict_tic['AAPL'])
data_dict_group_3_generated=training(data_dict_group[3])
GOOG_data_generated=training(GOOG_data)
data_all_generated=training(data_all)

In [ ]:
AAPL_res=[evaluation(data_dict_tic['AAPL'],data_dict_tic_APPL_generated)]
group_3_res=[evaluation(data_dict_group[3],data_dict_group_3_generated)]

In [ ]:
GOOG_data_res=[evaluation(GOOG_data,GOOG_data_generated)]
all_data_res=[evaluation(data_all,data_all_generated)]

In [ ]:
group_4_res=[evaluation(data_dict_group[4],data_dict_group_4_generated)]

## result analysis

In [ ]:
display(GOOG_data_res)

In [ ]:
display(AAPL_res)

In [ ]:
display(group_3_res)

In [ ]:
display(all_data_res)

In [ ]:
display(group_4_res)

In [ ]:
data_dict_tic_res={}
for tic in data_dict_tic_res.keys():
    display(data_dict_tic_res[tic])

In [ ]:
visualization(GOOG_processed_data,GOOG_genrated_data, 'pca')
visualization(GOOG_processed_data,GOOG_genrated_data, 'tsne')

In [ ]:
for k in data_dict.keys():
    print('vis of '+k)
    visualization(data_dict[k], generated_data_dict[k], 'pca')
    visualization(data_dict[k], generated_data_dict[k], 'tsne')

# Leverage Effect

In [ ]:
def get_std_list(data):
    intervals=get_intervals(data)
    std_list=[]
    data.drop(columns=['index'])
    for interval in intervals:
        data_seg=data.iloc[interval[0]:interval[1],:].pct_return.to_numpy()
        std=data_seg.std()
        std_list.append(std)
    return std_list

single stock

In [ ]:
for tic in tics:
    for j in range(regime_num):
        data=pd.read_csv('data_seg_'+tic+'_'+str(j)+'.csv')
        print('stock',tic,'regime',j)
        std_list=get_std_list(data)
        display(pd.DataFrame(std_list).describe()) 

by stock group

In [ ]:
for i in range(stock_group_num):
    for j in range(regime_num):
        data=pd.read_csv('data_seg_'+str(i)+'_'+str(j)+'.csv')
        print('stock',i,'regime',j)
        std_list=get_std_list(data)
        display(pd.DataFrame(std_list).describe()) 

In [ ]:
def get_std_list(data):
    intervals=get_intervals(data)
    std_list=[]
    data.drop(columns=['index'])
    for interval in intervals:
        data_seg=data.iloc[interval[0]:interval[1],:].to_numpy()
        std=data_seg.adj.std()
        std_list.append(std)
    return std_list

# key takeaway
1. regime 0 has high variance mean and high variance variance
2. regime 1 has low variance mean and low variance variance
3. regime 2 has low variance mean and low/high variance variance

# Pre-train Static learning classification discriminator

In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sktime.classification.kernel_based import RocketClassifier

In [ ]:
def get_data_of_same_length_df(data,seq_len):
    data_processed=[]
    intervals=get_intervals(data)
    temp_data=[]
    data.drop(columns=['index'])
    for interval in intervals:
        data_seg=data.iloc[interval[0]:interval[1],:]
        for i in range(0, len(data_seg) - seq_len):
            _x = data[i:i + seq_len]
            temp_data.append(_x)
    return temp_data

In [ ]:
data=pd.read_csv('data_seg_'+"0"+'_'+"0"+'.csv')
display(data.columns)

In [ ]:
for tic in tics:
    print(tic)
    X=[]
    y=np.empty(0)
    for j in range(regime_num):
        data=pd.read_csv('data_seg_'+tic+'_'+str(j)+'.csv').loc[:,['index', 'open', 'high', 'low', 'close', 'adjcp',
       'pct_return', 'adjcp_filtered', 'pct_return_filtered']]
        process_data=get_data_of_same_length_df(data,24)
        label=np.full(len(process_data), j)
        X.extend(process_data)
        y=np.concatenate((y, label), axis=0)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    classifier = RocketClassifier(num_kernels=2000,n_jobs=-1)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(accuracy_score(y_test, y_pred))

ALL single stock classification have unbelieve 100% acc?

In [ ]:
for i in range(stock_group_num):
    print('stock_group',i)
    X=[]
    y=np.empty(0)
    for j in range(regime_num):
        data=pd.read_csv('data_seg_'+str(i)+'_'+str(j)+'.csv').loc[:,['index', 'open', 'high', 'low', 'close', 'adjcp',
       'pct_return', 'adjcp_filtered', 'pct_return_filtered']]
        process_data=get_data_of_same_length_df(data,24)
        label=np.full(len(process_data), j)
        X.extend(process_data)
        y=np.concatenate((y, label), axis=0)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    classifier = RocketClassifier(num_kernels=2000,n_jobs=-1,use_multivariate='yes')
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(accuracy_score(y_test, y_pred))

Still 1.0?

In [ ]:

# X_train, X_test, y_train, y_test = train_test_split(X, y)
# classifier = TimeSeriesForestClassifier()
# classifier.fit(X_train, y_train)
# y_pred = classifier.predict(X_test)
# accuracy_score(y_test, y_pred)

# Pre-train Deep learning classification discriminator

### training

In [ ]:
from tsai.all import *
my_setup()

In [ ]:
for i in range(stock_group_num):
    print('stock_group',i)
    X=[]
    y=np.empty(0)
    for j in range(regime_num):
        data=pd.read_csv('data_seg_'+str(i)+'_'+str(j)+'.csv').loc[:,['index', 'open', 'high', 'low', 'close', 'adjcp',
       'pct_return', 'adjcp_filtered', 'pct_return_filtered']]
        process_data=get_data_of_same_length(data,24)
        label=np.full(len(process_data), j)
        X.extend([p.transpose() for p in process_data])
        y=np.concatenate((y, label), axis=0)
    X=np.array(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    X, y, splits = combine_split_data([X_train, X_test], [y_train, y_test])
    tfms  = [None, [Categorize()]]
    dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)
    dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64, 128], batch_tfms=[TSStandardize()], num_workers=0)
    model = InceptionTime(dls.vars, dls.c)
    learn = Learner(dls, model, metrics=accuracy)
    learn.fit_one_cycle(25, lr_max=1e-3)
    learn.plot_metrics()
    learn.save_all(path='export', dls_fname='dls_'+str(i)+'_'+str(j), model_fname='model_'+str(i)+'_'+str(j), learner_fname='learner_'+str(i)+'_'+str(j))
#     display(type(X_train),X_train.shape)

# Key takeaway

InceptionTime can do the job

In [ ]:
X, y, splits = get_classification_data('LSST', split_data=False)

In [ ]:
tfms  = [None, TSClassification()] # TSClassification == Categorize
batch_tfms = TSStandardize()
dls = get_ts_dls(X, new_y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=[64, 128])
dls.dataset

In [ ]:
def get_std_list(data):
    intervals=get_intervals(data)
    std_list=[]
    data.drop(columns=['index'])
    for interval in intervals:
        data_seg=data.iloc[interval[0]:interval[1],:].to_numpy()
        std=data_seg.adj.std()
        std_list.append(std)
    return std_list

In [ ]:
batch_tfms = TSStandardize(by_sample=True)
mv_clf = TSClassifier(X, y, splits=splits, path='models', arch=InceptionTimePlus, batch_tfms=batch_tfms, metrics=accuracy, cbs=ShowGraph())
mv_clf.fit_one_cycle(10, 1e-2)
mv_clf.export("mv_clf.pkl")

## inference

## inference